In [1]:
from bs4 import BeautifulSoup
import re

def get_text_from_p_elements(xml_data):
    try:
        # Create a BeautifulSoup object to parse the XML data
        soup = BeautifulSoup(xml_data, 'lxml')

        # Find all <p> elements and extract their text content
        text_list = [p.get_text(strip=True) for p in soup.find_all('p')]
        text_list = [text for text in text_list if not re.match(r'^\s*$', text)]

        return text_list

    except Exception as e:
        print("Error parsing the XML data:", e)
        return []

In [2]:
import json
sec_file = open("../data/dex106-TEI.xml")
sec_data = sec_file.read()
p_list = get_text_from_p_elements(sec_data)
p_list[:5]

/Users/amaechiabuah/Desktop/LEDGAR_provision_classification/.venv/lib/python3.10/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


['unknown',
 'Converted from a Word document',
 '$750,000,000 Three-Year Credit Agreement',
 'EX-10.6 6 dex106.htm $750,000,000 THREE-YEAR CREDIT AGREEMENT',
 'Exhibit 10.6']

In [3]:
import os
os.getcwd()

'/Users/amaechiabuah/Desktop/LEDGAR_provision_classification/classification'

In [4]:
from classification_baselines import *

labeled_set = {}
corpus_file = "../sec_corpus_2016-2019_freq100.jsonl"
dataset: SplitDataSet = split_corpus(corpus_file)
mlb = MultiLabelBinarizer().fit(dataset.y_train)
tfidfizer = TfidfVectorizer(sublinear_tf=True)
tfidfizer.fit_transform(dataset.x_train)


<810169x38861 sparse matrix of type '<class 'numpy.float64'>'
	with 44056231 stored elements in Compressed Sparse Row format>

In [18]:
import pickle
import json

l = {}
model = pickle.load(open("../saved_models/logreg_sec_corpus_2016-2019_freq100.pkl", "rb"))
p_vec = tfidfizer.transform(p_list)
y_vec = model.predict(p_vec)
l = {text:list(label) for label,text in zip(stringify_labels(y_vec, mlb), p_list)}
with open("out.json", 'w') as out:
    json.dump(l, out, indent=1, ensure_ascii=False)
#l   